# SQL-03. Соединение таблиц.

----

## Описание таблиц о футбольных матчах и командах.

---

Вы уже умеете делать запросы к одной таблице, использовать агрегатные функции и фильтровать данные в выводе. Но в реальных условиях базы данных обычно содержат множество таблиц и при запросе необходимо обращаться к нескольким таблицам. Освоением этого навыка мы сейчас и займёмся.

В этом модуле мы будем работать с таблицами о футбольных матчах и командах.

Таблицы этого модуля, как и все другие в курсе, лежат в схеме `sql` в [Metabase](http://sql.skillfactory.ru:3000/). Нам понадобятся таблицы `teams` и `matches`.


**Таблица** `teams` **с данными о командах**

| Название столбца | Содержимое столбца           |
|------------------|------------------------------|
| id               | id команды                   |
| api_id           | ключ на таблицу matches      |
| long_name        | полное название команды      |
| short_name       | сокращённое название команды |



**Таблица** `matches` **с данными о матчах**

| Название столбца | Содержимое столбца                             |
|------------------|------------------------------------------------|
| id               | id матча                                       |
| season           | сезон                                          |
| date             | дата матча                                     |
| home_team_api_id | api_id домашней команды, ключ на таблицу teams |
| away_team_api_id | api_id гостевой команды, ключ на таблицу teams |
| home_team_goals  | количество голов домашней команды              |
| away_team_goals  | количество голов гостевой команды              |

> **Задание  1.1**
>
> Сколько различных полных названий команд в таблице `teams`?

```sql
SELECT
    COUNT(DISTINCT long_name)
FROM
    sql.teams
```

**Ответ:** 296

> Сколько в таблице teams команд с коротким названием *VAL*?

```sql
SELECT
    COUNT(*)
FROM
    sql.teams
WHERE
    short_name = 'VAL'
```

**Ответ:** 3

> Информацию о скольких матчах содержит таблица `matches`?

```sql
SELECT
    COUNT(*)
FROM
    sql.matches
```

**Ответ:** 25083


> Данные за какие сезоны даны в таблице matches?
Ответ введите в формате 2019/2020.

```sql
SELECT
    DISTINCT season
FROM
    sql.matches
ORDER BY
    season
```

**Ответ:** 2008/2009 - 2015/2016

> **Задание 1.2**
> 
> Напишите запрос, который выведет сезон (`season`), а также общее количество забитых мячей домашними (`total_home_goals`) и гостевыми (`total_away_goals`) командами.
> Отсортируйте по столбцу с сезоном в порядке возрастания.


```sql
SELECT
    season,
    SUM(home_team_goals) AS total_home_goals,
    SUM(away_team_goals) AS total_away_goals
FROM
    sql.matches
GROUP BY
    season
ORDER BY
    season ASC
```

## Объединение таблиц без операторов

---

Чтобы соединить две таблицы между собой, достаточно записать названия таблиц через запятую в разделе `FROM`. Что произойдёт в таком случае?

```sql
SELECT *
FROM
    sql.teams,
    sql.matches
```

Каждая запись, которая есть в таблице `teams`, будет соединена с каждой записью в таблице `matches`.

Это действие также называют декартовым произведением таблиц.

Действительно ли это произведение?

Легко проверить! В исходных таблицах `teams` и `matches` было `299` и `25083` записей соответственно. Если соединить каждую запись одной таблицы с каждой записью другой, получится `299` `*` `25083` записей в итоговой таблице.

> **Задание 2.1**
>
> Напишите запрос, который выведет количество строк соединённой таблицы.


```sql
SELECT
    COUNT(*)
FROM
    sql.matches,
    sql.teams
```

В данном случае соединение таблиц не даёт практической пользы: мы получили очень много записей, которые никак не можем интерпретировать, потому что команды не соответствуют матчам.

Давайте исправим это. В таблице teams есть столбец `api_id`, а таблица `matches` содержит столбцы `home_team_api_id` и `away_team_api_id` — это ключи таблиц, по которым они соединяются.

**Ключ** — это поле (столбец) в таблице, которое позволяет однозначно идентифицировать запись (строку).

Чтобы соединить таблицы и получить данные о домашней команде по каждому матчу, добавим условие `where home_team_api_id = api_id`.

```SQL
SELECT
    *
FROM
    sql.teams,
    sql.matches
WHERE
    home_team_api_id = api_id
```

Аналогично можем получить данные о гостевых командах: необходимо изменить условие на
where `away_team_api_id = api_id`.

```SQL
SELECT
    *
FROM
    sql.teams,
    sql.matches
WHERE
    away_team_api_id = api_id
```

Итак, мы только что объединили таблицы по ключу.

Вы уже знакомы с ключами по таблице `pokemon` (там в этой роли выступал столбец `id`). Ключи нужны для того, чтобы иметь возможность не перепутать между собой различные записи.

Например, у нас есть несколько команд с одинаковым названием: *Polonia* *Bytom*, *Widzew* *Łódź* и *Royal* *Excel* *Mouscron* — хотя это разные команды, с разными `id`.

Кроме того, как мы уже смогли убедиться, ключи используются для соединения таблиц между собой.

**Ключи бывают двух основных типов:**

* *Primary* — первичный ключ — служит для идентификации текущей таблицы и, как правило, идёт первым в списке столбцов. Всегда уникален: повторяющихся значений в основной таблице быть не может.<br><br>
* *Foreign* — внешний ключ — представляет собой ссылку на другую таблицу.<br><br>

Как правило, названия ключей имеют «хвост», который позволяет их идентифицировать: например, `_id`, `_rk`, `_cd`, `_pk` (от `primary_key`), `_fk` (от `foreign_key`) и другие.

**Обратите внимание!** В данном датасете ключ `api_id` таблицы `teams` может быть использован в разных значениях. Его можно использовать для того, чтобы получить информацию о домашней (`home`) или гостевой (`away`) команде.

Вы могли заметить, что в последних двух запросах получилось очень много столбцов. Как и при работе с одиночной таблицей, мы можем выбирать, какие столбцы соединённой таблицы выводить.

С помощью известного нам запроса получим названия команд, игравших домашние матчи, и счёт матчей.


```sql
SELECT
    long_name,
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE
    home_team_api_id = api_id
```

> **Задание 2.2**
>
> Напишите запрос, который выведет таблицу с результатами матчей для гостевых команд, содержащую:
>
> * названия гостевых команд (`long_name`),<br><br>
> * количество забитых мячей домашней команды (`home_team_goals`),<br><br>
> * количество забитых мячей гостевой команды (`away_team_goals`).<br><br>


```sql
SELECT
    long_name,
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE
    away_team_api_id = api_id
```

## JOIN

---

В прошлом юните для соединения таблиц мы использовали условие в разделе `WHERE`, чтобы показать принцип работы оператора `JOIN`.

В качестве примера используем запрос из предыдущего юнита.


```SQL
SELECT
    long_name,
    home_team_goals,
    away_team_goals
FROM
    sql.teams,
    sql.matches
WHERE
    home_team_api_id = api_id
```

и запишем его с использованием `JOIN`.


```sql
SELECT
    long_name,
    home_team_goals,
    away_team_goals
FROM
    sql.teams
    JOIN sql.matches ON home_team_api_id = api_id
```

> **Задание 3.1**
>
> Выполните данный запрос


```sql
SELECT
    *
FROM
    sql.teams,
    sql.matches
WHERE
    away_team_api_id = api_id
```

> с использованием оператора `JOIN`, исключив оператор `WHERE`.


```sql
SELECT
    *
FROM
    sql.teams
    JOIN sql.matches ON away_team_api_id = api_id
```

**СИНТАКСИС**

Оператор `JOIN` упрощает процесс соединения таблиц.

Его синтаксис можно представить следующим образом:


```sql
SELECT
    столбец1,
    столбец2,
    ...
FROM
    таблица1
    JOIN таблица2 ON условие
```

Порядок присоединения таблиц в данном случае не важен — результат будет одинаковым.

С помощью `JOIN` можно соединить и более двух таблиц.


```sql
SELECT
    столбец1,
    столбец2,
    ...
FROM
    таблица1
    JOIN таблица2 ON условие
    JOIN таблица3 ON условие
```

В таблицах, которые мы соединяем, могут быть одинаковые названия столбцов.

К примеру, столбец `id` есть и в таблице `matches`, и в таблице `teams`. Такой запрос не будет обработан.


```sql
SELECT
    id
FROM
    sql.teams
    JOIN sql.matches ON home_team_api_id = api_id
```

В результате должно появиться сообщение об ошибке вроде такого: *"... column id is ambiguous ..."*.

Что же делать в таком случае?

Можно указать, откуда мы хотим запросить данные, записав название таблицы перед столбцом через точку.


```sql
SELECT
    teams.id
FROM
    sql.teams
    JOIN sql.matches ON home_team_api_id = api_id
```

Можно также выбрать и столбец из таблицы `matches`.



```sql
SELECT
    matches.id
FROM
    sql.teams
    JOIN sql.matches on home_team_api_id = api_id
```

Зачастую названия таблиц слишком длинные, так что использовать их неудобно.

Упростить обращение к различным таблицам можно, присвоив им сокращённые названия — **алиасы** (от англ. *alias*).

Синтаксис для указания алиаса такой же, как и для названия столбца.


```sql
SELECT
    столбец1,
    столбец2,
    ...
FROM
    таблица1 AS короткое_название_1
    JOIN таблица2 AS короткое_название_2 ON условие
```

Название записывается без пробелов и операторов.

Если необходимо записать название, в котором используются пробелы ("table 1"), то алиас можно обернуть в кавычки.

**Важно!** Обращаться по такому алиасу придётся также с помощью кавычек.


```sql
SELECT
    "table 1".столбец1,
    "table 2".столбец2,
    ...
FROM
    таблица1 AS "table 1"
    JOIN таблица2 AS "table 2" ON условие
```

**Использование таких алиасов считается плохой практикой как минимум по причине того, что обращаться с такими алиасами неудобно.**

Вместо алиасов с кавычками рекомендуется давать простое короткое название на латинице, без специальных символов и пробелов. Если таблиц немного и все названия начинаются с разных букв, можно присваивать алиасы по первой букве.

Ключевое слово `as`, как и в названии столбца, можно опустить в большинстве СУБД.


```sql
SELECT
    столбец1 новое_название_столбца,
    столбец2 новое_название_столбца,
    ...
FROM
    таблица1 короткое_название_1
    JOIN таблица2 короткое_название_2 ON условие
```

> **Задание 3.2**
>
> Напишите запрос, который выведет два столбца: id матча (`match_id`) и id домашней команды (`team_id`). Отсортируйте по id матча в порядке возрастания значений.


```sql
SELECT
    matches.id AS match_id,
    teams.id AS team_id
FROM
    sql.matches
    JOIN sql.teams ON home_team_api_id = api_id
ORDER BY
    match_id ASC
```

Кажется, теперь мы можем соединить таблицы между собой и создать своё табло со счётом матчей, как на спортивных сайтах!

Давайте с помощью запроса SQL получим таблицу, содержащую:

* название домашней команды;<br><br>
* количество забитых домашней командой голов;<br><br>
* количество забитых гостевой командой голов;<br><br>
* название гостевой команды.<br><br>

```sql
SELECT
    h.long_name "домашняя команда",
    m.home_team_goals "голы домашней команды",
    m.away_team_goals "голы гостевой команды",
    a.long_name "гостевая команда"
FROM
    sql.matches m
    JOIN sql.teams h ON m.home_team_api_id = h.api_id
    JOIN sql.teams a ON m.away_team_api_id = a.api_id
```

> **Задание 3.3**
>
> Напишите запрос, который выведет столбцы:
>
> * id матча,
> * короткое название домашней команды (home_short),
> * короткое название гостевой команды (away_short).
>
> Отсортируйте запрос по возрастанию id матча.


```sql
SELECT
    m.id,
    h.short_name AS home_short,
    a.short_name AS away_short
FROM
    sql.matches AS m
    JOIN sql.teams AS h ON m.home_team_api_id = h.api_id
    JOIN sql.teams AS a ON m.away_team_api_id = a.api_id
ORDER BY
    m.id
```

## Фильтрация и агрегатные функции

---

Соединять таблицы мы научились, теперь давайте научимся получать необходимые данные из нескольких таблиц.

Принцип построения запроса и порядок операторов такой же, как и с обычной таблицей.

Вспомним его:


```sql
SELECT... 
FROM... 
WHERE... 
GROUP BY... 
ORDER BY... 
LIMIT...
```

**Фильтрация данных**

К соединённым таблицам применимы функции фильтрации данных.

Например, можно вывести *id* матчей, в которых команда *Arsenal* была гостевой.

```sql
SELECT
    m.id
FROM
    sql.teams t
    JOIN sql.matches m ON m.away_team_api_id = t.api_id
WHERE
    long_name = 'Arsenal'
```

Принципиальное отличие фильтрации данных по соединённым таблицам от аналогичного действия по одиночным таблицам заключается в том, что, фильтруя записи одной таблицы, мы также будем фильтровать и записи другой таблицы, поскольку соединённые на уровне запроса таблицы по сути являются единой таблицей.

Например, результат запроса


```sql
SELECT
    m.id id_1,
    m.season,
    t.id id_2,
    t.long_name
FROM
    sql.teams t
    JOIN sql.matches m ON m.away_team_api_id = t.api_id
```

можно разделить на две разные части


![](data/dst3-u2-md3_4_1.png)

Одна часть — таблица `matches` с алиасом `m`, вторая — `teams` с алиасом `t`, но после соединения они являются одной таблицей.

Таким образом, если вы отфильтруете данные по одной части таблицы, то другая, соединённая, часть пропадёт вместе с ней.

> **Задание 4.1**
> 
> Напишите запрос, который выведет полное название домашней команды (`long_name`), количество голов домашней команды (`home_goal`) и количество голов гостевой команды (`away_goal`) в матчах, где домашней командой были команды с коротким названием 'GEN'.
> Отсортируйте запрос по id матча в порядке возрастания.


```sql
SELECT
    t.long_name,
    m.home_team_goals AS home_goal,
    m.away_team_goals AS away_goal
FROM
    sql.teams AS t
    JOIN sql.matches AS m ON t.api_id = m.home_team_api_id
WHERE
    t.short_name = 'GEN'
ORDER BY
    m.id ASC
```

Также мы можем отфильтровать записи сразу по двум таблицам.

Например, можно оставить только записи, в которых короткое название домашней команды `GEN` и матчи сезона `2008/2009`.

```sql
SELECT
    *
FROM
    sql.matches m
    JOIN sql.teams t on t.api_id = m.home_team_api_id
WHERE
    t.short_name = 'GEN'
    AND m.season = '2008/2009'
```

> **Задание 4.2**
> 
> Напишите запрос, чтобы вывести `id` матчей, короткое название домашней команды (`home_short`), короткое название гостевой команды (`away_short`) для матчей сезона `2011`/`2012`, в которых участвовала команда с названием `Liverpool`. Отсортируйте по `id` матча в порядке возрастания.


```sql
SELECT
    m.id,
    h.short_name AS home_short,
    a.short_name AS away_short
FROM
    sql.matches AS m
    JOIN sql.teams AS h ON m.home_team_api_id = h.api_id
    JOIN sql.teams AS a ON m.away_team_api_id = a.api_id
WHERE
    m.season = '2011/2012'
    AND (
        h.long_name = 'Liverpool'
        OR a.long_name = 'Liverpool'
    )
ORDER BY
    m.id
```

**Агрегация данных**

К соединённым таблицам также применимы любые агрегатные функции — самые важные функции для анализа данных.

Например, мы можем вывести сумму голов матча, забитых командами, агрегированную по гостевым командам (совокупное количество голов в матче, забитых обеими командами, суммированное в разрезе гостевых команд).


```sql
SELECT
    t.long_name,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals
FROM
    sql.matches m
    JOIN sql.teams t ON m.away_team_api_id = t.api_id
GROUP BY
    t.id
```

**Обратите внимание!** В данном запросе была использована группировка по столбцу `id` таблицы `teams`, хотя этот столбец не выводится в запросе. Это необходимо для того, чтобы команды с одинаковым названием, если такие найдутся, не группировались между собой. Группировка по названию команды в данном запросе будет неверной, так как есть несколько команд с одинаковым полным названием — мы говорили об этом в начале модуля.

Также, применяя агрегатные функции к соединённым таблицам, обращайте внимание на указание алиасов (или таблиц) при группировке и указании столбцов агрегатных функций. В нашей соединённой таблице есть два столбца с названием `id`, и если бы мы сформировали запрос без указания таблицы, как указано ниже, то...


```sql
SELECT
    t.long_name,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals
FROM
    sql.matches m
    JOIN sql.teams t ON m.away_team_api_id = t.api_id
GROUP BY
    id
```

система выдала бы уже знакомую нам ошибку: *"... column "id" is ambiguous ..."*

Мы можем использовать оператор `HAVING` для фильтрации сгруппированных данных.

Поставим задачу — вывести таблицу с суммарным количеством забитых голов в матчах по командам и сезонам для команд, в которых суммарное количество голов в матчах сезона больше 100.

Для начала посчитаем общее количество голов в матчах по сезонам.


```sql
SELECT
    m.season,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) AS total_goals
FROM 
    sql.matches AS m
GROUP BY
    m.season
```

Затем добавим таблицу с командами, группировку по командам и условие фильтрации.


```sql
SELECT
    m.season,
    t.long_name,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) AS total_goals
FROM
    sql.matches AS m
    JOIN sql.teams AS t ON t.api_id = m.home_team_api_id
    OR t.api_id = m.away_team_api_id
GROUP BY
    m.season,
    t.id
HAVING
    SUM(m.home_team_goals) + SUM(m.away_team_goals) > 100
```

**Проверка кода для команды Aberdeen (api_id=8485) в сезоне 2015/2016**

* Код с платформы (с добавлением фильтрации и столбца api_id)


```sql
SELECT
    m.season,
    t.long_name,
    t.api_id,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) AS total_goals
FROM
    sql.matches AS m
    JOIN sql.teams AS t ON t.api_id = m.home_team_api_id
    OR t.api_id = m.away_team_api_id
GROUP BY
    m.season,
    t.id
HAVING
    SUM(m.home_team_goals) + SUM(m.away_team_goals) > 100 
    AND t.api_id = '8485'
    AND m.season='2015/2016'`
```

**Ответ: 110**


* Сумма забитых голов в гостях


```sql
SELECT
    m.season,
    t.long_name,
    SUM(away_team_goals) AS total_away_goals
FROM
    sql.matches AS m
    JOIN sql.teams AS t ON m.away_team_api_id = t.api_id
GROUP BY
    m.season,
    m.away_team_api_id,
    t.id
HAVING
    away_team_api_id = '8485' AND season = '2015/2016'
```

**Ответ: 32**



* Сумма забитых голов дома


```sql
SELECT
    m.season,
    t.long_name,
    SUM(home_team_goals) AS total_home_goals
FROM
    sql.matches AS m
    JOIN sql.teams AS t ON m.home_team_api_id = t.api_id
GROUP BY
    m.season,
    m.home_team_api_id,
    t.id
HAVING
    home_team_api_id = '8485' AND season = '2015/2016'
```

**Ответ: 30**

* Код, который помог лучше понять как по заданному условию в таблицу `matches` подтягиваются данные из таблицы `teams`, и как затем происходит группировка:


```sql
SELECT
    season,
    t.api_id,
    m.home_team_api_id,
    m.away_team_api_id,
    SUM(m.home_team_goals) AS total_home,
    SUM(m.away_team_goals) AS total_away,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) AS total_goals
FROM
    sql.matches AS m
    JOIN sql.teams AS t ON (
        t.api_id = m.home_team_api_id
        OR t.api_id = m.away_team_api_id
    )
GROUP BY
    m.season,
    t.api_id,
    m.home_team_api_id,
    m.away_team_api_id
HAVING
    t.api_id = '8485'
    AND season = '2015/2016'
```

**Вывод:** в задаче речь шла о сумме забитых и пропущенных голов командами за сезон, а не только забитых, как я изначально подумал.

* Код, который считает сумму забитых и пропущенных за сезон для тестовой команды:


```sql
SELECT
    SUM(home_team_goals) + SUM(away_team_goals)
FROM
    sql.matches
WHERE
    (
        home_team_api_id = '8485'
        OR away_team_api_id = '8485'
    )
    AND season = '2015/2016'
```

@Андрей_Кукунов_ментор 

Здравствуйте, Андрей!

Во-первых, не понимаю (хотя догадываюсь), зачем Вы начали новый тред! Изначально вопрос был по конкретной задаче на платформе, так что от темы я не отклонялся. Поэтому продолжаю обсуждение здесь.

Во-вторых, категорически не согласен с [озвученным Вами мнением](https://app.pachca.com/chats/4734580?message=77525718), что неточность формулировки - это дескать *"педагогический приём, стимулирующий студентов на поиск смыслов"*. Когда пытаешься вникнуть в соединение таблиц, а особенно в группировку, совершенно не до поиска смыслов в условиях задачи. Это только создает  кумулятивное непонимание. Поэтому не должно быть (а в действительности и нет) таких приёмов! По крайней мере, в учебном материале, изучая который, студенты осмысливают механизм выполнения инструкций ЯП. Формулировки должны быть предельно точными и лаконичными и уж точно не содержать ошибок (убежден, что в данном случае со стороны авторов совершенно ненамеренных)!  

Соответственно, если при выполнении обсуждаемой задачи не учитывать сделанных якобы в учебных целях семантических ошибок в её условии,  то суммировать необходимо именно количество **забитых** командой голов за сезон (и в гостях, и дома). 

В этом случае код решения выглядел бы примерно так:


```sql
SELECT
    m.season,
    t.long_name,
    t.api_id,
    SUM(m.home_team_goals) + away_scored.total_away_scored_goals AS total_scored_goals
FROM
    sql.matches AS m
    JOIN sql.teams AS t ON t.api_id = m.home_team_api_id
    JOIN (
        --таблица забитых голов в гостях
        SELECT
            m.season,
            t.long_name,
            t.api_id,
            SUM(m.away_team_goals) AS total_away_scored_goals
        FROM
            sql.matches AS m
            JOIN sql.teams AS t ON t.api_id = m.away_team_api_id
        GROUP BY
            m.season,
            t.id
    ) AS away_scored ON (
        m.season = away_scored.season
        AND t.api_id = away_scored.api_id
    )
GROUP BY
    m.season,
    t.id,
    away_scored.total_away_scored_goals
HAVING
    SUM(m.home_team_goals) + away_scored.total_away_scored_goals > 100
```

**Может быть в программировании я мало смыслю, но вот на выверении формулировок (и кое на чём ещё) за последние 15 лет собаку съел.  Либо надо менять условие задачи, либо - код её решения, выложенный на платформе!**

> **Задание 4.3**
> 
> Напишите запрос, с помощью которого можно вывести список полных названий команд, сыгравших в гостях 150 и более матчей. Отсортируйте список по названию команды.


```sql
SELECT
    t.long_name
FROM
    sql.matches AS m
    JOIN sql.teams AS t ON m.away_team_api_id = t.api_id
GROUP BY
    m.away_team_api_id,
    t.long_name
HAVING
    count(m.away_team_api_id) >= 150
ORDER BY
    t.long_name
```

## Способы соединения таблиц

---

В прошлых юнитах для соединения таблиц мы использовали один оператор — `JOIN`. В действительности же он не один: существует несколько различных видов соединений (join’ов) — давайте посмотрим, какие 

### INNER JOIN

---

![](data/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-u2-md3_5_1.gif)

Для `INNER JOIN` работает следующее правило: присоединяются только те строки таблиц, которые удовлетворяют условию соединения. Если в любой из соединяемых таблиц находятся такие строки, которые не удовлетворяют заявленному условию, — они отбрасываются.

Рассмотрим на примере нашего датасета. 

В таблице `teams` есть данные о 299 различных командах — можем проверить это с помощью запроса.


```sql
SELECT
    COUNT(DISTINCT id)
FROM
    sql.teams
```

или


```sql
SELECT
    COUNT(DISTINCT api_id)
FROM
    sql.teams
```

Теперь добавим к teams таблицу с матчами.


```sql
SELECT
    COUNT(DISTINCT t.id)
FROM
    sql.teams t
    JOIN sql.matches m ON (
        t.api_id = m.home_team_api_id
        OR t.api_id = m.away_team_api_id
    )
```

И в таблице останется уже не 299 команд, а только 292.

Дело в том, что таблица `sql.matches` по какой-то причине не содержит информацию о командах `Lierse` `SK`, `KVC` `Westerlo`, `KAS` `Eupen`, `Club` `Brugge` `KV`, `KV` `Oostende`, `RSC` `Anderlecht` и `Hull` `City`, зато они есть в таблице `sql.teams`. Возможно, эти команды не участвовали ни в одном матче или записи по этим матчам были удалены.

### LEFT OUTER JOIN и RIGHT OUTER JOIN

---

Также существуют схожие друг с другом типы соединения — `LEFT JOIN` и `RIGHT JOIN` (слово outer в операторе можно опустить).

Принцип работы оператора можно описать так:

![](data/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-u2-md3_5_2.gif)

Для `LEFT JOIN` работает следующее правило: из левой (относительно оператора) таблицы сохраняются все строки, а из правой добавляются только те, которые соответствуют условию соединения. Если в правой таблице не находится соответствия, то значения строк второй таблицы будут иметь значение `NULL`.

`LEFT JOIN` может быть полезен, когда соответствующих записей во второй таблице может не быть, но важно сохранить записи из первой таблицы.

Поставим следующую задачу: вывести полные названия команд, данных по которым нет в таблице `matches`.

Для начала посмотрим на результат запроса после соединения.


```sql
SELECT
    t.long_name,
    m.id
FROM
    sql.teams t
    LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id
    OR t.api_id = m.away_team_api_id
ORDER BY
    m.id DESC
```

Вывод: в таблице `teams` сохранились все записи, а в таблице `matches` есть пустые строки.

Теперь, чтобы выбрать такие команды, которые не принимали участия в матчах, достаточно добавить условие where m.id is null (или любое другое поле таблицы matches).


```sql
SELECT
    t.long_name
FROM
    sql.teams t
    LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id
    OR t.api_id = m.away_team_api_id
WHERE
    m.id IS NULL
```

**Обратите внимание!** Если мы добавим какой-либо фильтр по отличному от `NULL` значению для таблицы matches, то `LEFT` `JOIN` превратится в `INNER` `JOIN`, поскольку для второй таблицы станет необходимым присутствие такого (`NOT` `NULL`) значения в строке.


```sql
SELECT
    t.long_name
FROM
    sql.teams t
    LEFT JOIN sql.matches m ON (
        t.api_id = m.home_team_api_id
        OR t.api_id = m.away_team_api_id
    )
WHERE
    m.season = '2008/2009'
    AND t.long_name = 'KAS Eupen'
```

В ответе Metabase получим `No results`!, так как вместе с фильтром m.season = '2008/2009' исчезли все строки, значения которых NULL.

> **Задание 5.1**
> 
> Используя LEFT JOIN, выведите список уникальных названий команд, содержащихся в таблице matches. Отсортируйте список в алфавитном порядке.

```sql
SELECT
    DISTINCT t.long_name
FROM
    sql.matches AS m
    JOIN sql.teams AS t ON (
        m.home_team_api_id = t.api_id
        OR m.away_team_api_id = t.api_id
    )
ORDER BY
    t.long_name
```

С `LEFT JOIN` также работают агрегатные функции, что позволяет не потерять значения из левой таблицы. Например, мы можем вывести сумму голов команд по гостевым матчам.


```sql
SELECT
    t.long_name,
    SUM(m.away_team_goals) total_goals
FROM
    sql.teams t
    LEFT JOIN sql.matches m ON t.api_id = m.away_team_api_id
GROUP BY
    t.id
ORDER BY
    2 DESC
```

**Обратите внимание!** При применении функций `SUM`, `MIN`, `MAX`, `AVG` к полям со значением `NULL` в результате получится `NULL`, а не 0. А при использовании функции `COUNT`, наоборот, получится 0.

> **Задание 5.2**
> 
> Используя `LEFT JOIN`, напишите запрос, который выведет полное название команды (`long_name`), количество матчей, в которых участвовала команда, — домашних и гостевых (`matches_cnt`). Отсортируйте по количеству матчей в порядке возрастания, затем — по названию команды в алфавитном порядке.


```sql
SELECT
    t.long_name,
    COUNT(m.away_team_api_id) AS matches_cnt
FROM
    sql.teams AS t
    LEFT JOIN sql.matches AS m ON (
        t.api_id = m.home_team_api_id
        OR t.api_id = m.away_team_api_id
    )
GROUP BY
    t.api_id,
    t.long_name
ORDER BY
    matches_cnt,
    t.long_name
```

При использовании `RIGHT JOIN` сохраняется та же логика, что и для `LEFT JOIN`, только за основу берётся правая таблица.

Чтобы из `LEFT JOIN` получить `RIGHT JOIN`, нужно просто поменять порядок соединения таблиц.

Вообще, применение `RIGHT JOIN` считается плохим тоном, так как язык `SQL` читается и пишется слева направо, а такой оператор усложняет чтение запросов.

### FULL OUTER JOIN

---

Оператор `FULL` `OUTER` `JOIN` объединяет в себе `LEFT` и `RIGHT` `JOIN` и позволяет сохранить кортежи обеих таблиц. Даже если не будет соответствий, мы сохраним все записи из обеих таблиц.

`FULL OUTER JOIN` может быть полезен в ситуациях, когда схема данных недостаточно нормализована и не хватает таблиц-справочников.

`Пример:` в базе данных интернет-магазина есть две таблицы — с зарегистрированными пользователями и пользователями, оформившими заказ. При этом оформить заказ можно без регистрации, а зарегистрироваться — без оформления заказа.

Предположим, что вам необходимо получить полный список пользователей — и оформивших заказ, и зарегистрированных, — но в базе данных этой объединённой таблицы нет. В данном случае можно использовать `FULL OUTER JOIN` для получения полного списка, соединив таким образом таблицы c заказами и регистрациями по `id` пользователя.

![](data/dst3-u2-md3_5_4.png)

Синтаксис `FULL OUTER JOIN` аналогичен другим `JOIN`.


```sql
SELECT
    …
FROM
    table1 FULL
    OUTER JOIN table2 ON условие
```

### CROSS JOIN

---

На самом деле с этим оператором соединения таблиц вы познакомились в самом начале текущего модуля.

`CROSS JOIN` соединяет таблицы так, что каждая запись в первой таблице присоединяется к каждой записи во второй таблице, иначе говоря, даёт декартово произведение.

![](data/dst3-u2-md3_5_5.png)

В начале модуля вы использовали этот оператор, записывая таблицы через запятую.


```sql
SELECT 
    *
FROM
    sql.teams,
    sql.matches
```

Этот же запрос можно записать с использованием `CROSS JOIN`.


```sql
SELECT 
    *
FROM
    sql.teams
    CROSS JOIN sql.matches
```

Также этот запрос можно записать с помощью `INNER JOIN` с условием `ON TRUE` — результат будет тот же.


```sql
SELECT 
    *
FROM
    sql.teams
    JOIN sql.matches ON TRUE
```

`CROSS JOIN` может быть полезен, когда необходимо создать таблицу фактов.

Например, с помощью такого запроса мы можем получить все возможные комбинации полных названий команд в матчах.


```sql
SELECT
    DISTINCT t1.long_name home_team,
    t2.long_name away_team
FROM
    sql.teams t1
    CROSS JOIN sql.teams t2
```

> **Задание 5.3**
> 
> Напишите запрос, который выведет все возможные уникальные комбинации коротких названий домашней команды (`home_team`) и коротких названий гостевой команды (`away_team`). Отсортируйте запрос по первому и второму столбцам.


```sql
SELECT
    DISTINCT t1.short_name home_team,
    t2.short_name away_team
FROM
    sql.teams AS t1
    CROSS JOIN sql.teams AS t2
ORDER BY
    1,
    2
```

### NATURAL JOIN

---

Ключевое слово `NATURAL` в начале оператора `JOIN` позволяет не указывать условие соединения таблиц — для соединения будут использованы столбцы с одинаковым названием из этих таблиц.

`NATURAL JOIN` можно использовать с любыми видами соединений, которые требуют условия соединения:

* `NATURAL INNER JOIN` (возможна запись `NATURAL JOIN)`;
* `NATURAL LEFT JOIN`;
* `NATURAL RIGHT JOIN`;
* `NATURAL FULL OUTER JOIN`.

При использовании `NATURAL JOIN` прежде всего стоит обратить внимание на ключи таблиц. Для наших таблиц `teams` и `matches` этот вид соединения не подойдёт, так как общим для обеих таблиц является столбец `id`, но таблицы соединяются по другим столбцам.

Когда у таблиц есть несколько столбцов с одинаковыми именами, при `NATURAL JOIN` условие соединения будет применено на все столбцы с одинаковыми именами.

### Общая логика построения запроса с JOIN
---

При построении запроса с несколькими `JOIN` старайтесь идти слева направо. Сначала выберите таблицу, которая является центральной в соответствии с поставленной задачей, вопросом. Затем добавляйте таблицы поэтапно в зависимости от бизнес-логики запроса.

Например, для ответа на вопрос: «Какая команда сыграла больше всех матчей в сезоне 2010/2011?» в качестве центральной лучше выбрать таблицу с командами.

А для ответа на вопрос: «В каком сезоне участвовало больше всего команд?» — таблицу с матчами.

Стоит отметить, что из рассмотренных видов соединений чаще всего используются `INNER JOIN` и `LEFT JOIN`. Другие операторы используются реже, но стоит помнить об их существовании при решении нестандартных задач.



## Виды JOINов (коротко)
---

**CROSS JOIN**

Cоединяет таблицы так, что каждая запись в первой таблице присоединяется к каждой записи во второй таблице ― иначе говоря, даёт декартово произведение.

**INNER JOIN**

— это тот же `JOIN` (слово `INNER` в операторе можно опустить).

Для `INNER JOIN` работает следующее правило: присоединяются только те строки таблиц, которые удовлетворяют условию соединения. Если в любой из соединяемых таблиц находятся такие строки, которые не удовлетворяют заявленному условию, — они отбрасываются.

**NATURAL JOIN**

Ключевое слово `NATURAL` в начале оператора `JOIN` позволяет не указывать условие соединения таблиц — для соединения будут использованы столбцы с одинаковым названием из этих таблиц.

`NATURAL JOIN` можно использовать с любыми видами соединений, которые требуют определённого условия:

* `NATURAL INNER JOIN` (возможна запись NATURAL JOIN);
* `NATURAL LEFT JOIN`;
* `NATURAL RIGHT JOIN`;
* `NATURAL FULL OUTER JOIN`.

При использовании `NATURAL JOIN` прежде всего стоит обратить внимание на ключи таблиц.

Когда у таблиц есть несколько столбцов с одинаковыми именами, при `NATURAL JOIN` условие соединения будет применено на все столбцы с одинаковыми именами.

**LEFT JOIN**

Для `LEFT JOIN` работает следующее правило: из левой (относительно оператора) таблицы сохраняются все строки, а из правой добавляются только те, которые соответствуют условию соединения. Если в правой таблице не находится соответствия, то значения строк второй таблицы будут иметь значение `NULL`.

`LEFT JOIN` полезен, когда соответствующих записей во второй таблице может не быть, но важно сохранить записи из первой таблицы.

Чтобы из `LEFT JOIN` получить `RIGHT JOIN`, нужно просто поменять порядок соединения таблиц.

**RIGHT JOIN**

Для `RIGHT JOIN` работает следующее правило: из правой (относительно оператора) таблицы сохраняются все строки, а из левой добавляются только те, которые соответствуют условию соединения. Если в левой таблице не находится соответствия, то значения строк второй таблицы будут иметь значение `NULL`.

`RIGHT JOIN` полезен, когда соответствующих записей во второй таблице может не быть, но важно сохранить записи из первой таблицы.

Чтобы из `RIGHT JOIN` получить LEFT JOIN, нужно просто поменять порядок соединения таблиц.

Вообще, применение `RIGHT JOIN` считается дурным тоном, так как язык *SQL* читается и пишется слева направо, и этот оператор усложняет чтение запросов.

**FULL OUTER JOIN**

Оператор `FULL OUTER JOIN` объединяет в себе `LEFT` и `RIGHT JOIN` и позволяет сохранить кортежи обеих таблиц. Даже если не будет соответствий, мы сохраним все записи из обеих таблиц.

`FULL OUTER JOIN` может быть полезен в ситуациях, когда схема данных недостаточно нормализована и не хватает таблиц-справочников.

Пример: в базе данных интернет-магазина есть две таблицы — с зарегистрированными пользователями и пользователями, оформившими заказ. При этом оформить заказ можно без регистрации, а зарегистрироваться — без оформления заказа.

Предположим, что вам необходимо получить полный список пользователей — и оформивших заказ, и зарегистрированных, — но в базе данных этой объединённой таблицы нет. В данном случае можно использовать `FULL OUTER JOIN` для получения полного списка, соединив таким образом таблицы c заказами и регистрациями по `id `пользователя.

## Практические задания
---

### Задание 6.1

> Напишите запрос, который выведет список уникальных полных названий команд (`long_name`), игравших в гостях в матчах сезона 2012/2013. Отсортируйте список в алфавитном порядке.

**Вариант 1** 

```sql
SELECT
    t.long_name
FROM
    sql.matches AS m
    JOIN sql.teams AS t ON (
        m.away_team_api_id = t.api_id
        AND m.season = '2012/2013'
    )
GROUP BY
    t.long_name
ORDER BY
    t.long_name
```

**Вариант 2** (выводит список команд, сыгравших хотя бы один матч в сезоне)

```sql
SELECT 
    t.long_name
FROM
    sql.teams AS t
    JOIN sql.matches AS h ON (
        t.api_id = h.home_team_api_id
        AND h.season = '2012/2013'
    )
    JOIN sql.matches AS a ON (
        t.api_id = a.away_team_api_id
        AND a.season = '2012/2013'
    )
GROUP BY
    t.long_name
ORDER BY
    t.long_name
```

**Эталонное решение**

```sql
SELECT
    DISTINCT t.long_name
FROM
    sql.teams t
    JOIN sql.matches m ON t.api_id = m.away_team_api_id
WHERE
    m.season = '2012/2013'
ORDER BY
    1
```

### Задание 6.2

> Напишите запрос, который выведет полное название команды (`long_name`) и общее количество матчей (`matches_cnt`), сыгранных командой `Inter` в домашних матчах.

```sql
SELECT
    t.long_name,
    COUNT(t.long_name) AS 'matches_cnt'
FROM
    sql.teams AS t
    JOIN sql.matches AS m ON (
        m.home_team_api_id = t.api_id
        AND t.long_name = 'Inter'
    )
GROUP BY
    t.long_name
```

**Эталонное решение**

```sql
SELECT
    t.long_name,
    COUNT(m.id) matches_cnt
FROM
    sql.matches m
    JOIN sql.teams t ON t.api_id = m.home_team_api_id
WHERE
    t.long_name = 'Inter'
GROUP BY
    t.id
```

### Задание 6.3

> Напишите запрос, который выведет топ-10 команд (`long_name`) по суммарному количеству забитых голов в гостевых матчах. Во втором столбце запроса выведите суммарное количество голов в гостевых матчах (`total_goals`).

```sql
SELECT
    t.long_name,
    SUM(m.away_team_goals) AS total_goals
FROM
    sql.teams AS t
    JOIN sql.matches AS m ON m.away_team_api_id = t.api_id
GROUP BY
    t.api_id,
    t.long_name
ORDER BY
    total_goals DESC
LIMIT
    10
```

**Эталонное решение**

```sql
SELECT
    t.long_name,
    SUM(m.away_team_goals) total_goals
FROM
    sql.teams t
    JOIN sql.matches m ON t.api_id = m.away_team_api_id
GROUP BY
    t.id
ORDER BY
    2 desc
LIMIT
    10
```

### Задание 6.4

> Выведите количество матчей между командами `Real Madrid CF` и `FC Barcelona`. В поле ниже введите запрос, с помощью которого вы решили задание.

```sql
SELECT
    COUNT(*)
FROM
    sql.matches as m
    JOIN sql.teams as t1 ON m.home_team_api_id = t1.api_id
    JOIN sql.teams as t2 ON m.away_team_api_id = t2.api_id
WHERE
    (
        t1.long_name = 'Real Madrid CF'
        AND t2.long_name = 'FC Barcelona'
    )
    OR (
        t1.long_name = 'FC Barcelona'
        AND t2.long_name = 'Real Madrid CF'
    )
```

**Эталонное решение**

```sql
SELECT
    COUNT (*)
FROM
    sql.matches m
    JOIN sql.teams h ON h.api_id = m.home_team_api_id
    JOIN sql.teams a ON a.api_id = m.away_team_api_id
WHERE
    (
        h.long_name = 'Real Madrid CF'
        and a.long_name = 'FC Barcelona'
    )
    or (
        a.long_name = 'Real Madrid CF'
        and h.long_name = 'FC Barcelona'
    )
```

### Задание 6.5

> Напишите запрос, который выведет название команды (`long_name`), сезон (`season`) и суммарное количество забитых голов в домашних матчах (`total_goals`). Оставьте только те строки, в которых суммарное количество голов менее десяти. Отсортируйте запрос по названию команды, а затем — по сезону.

```sql
SELECT
    t.long_name,
    m.season,
    SUM(m.home_team_goals) AS total_goals
FROM
    sql.matches AS m
    JOIN sql.teams AS t ON m.home_team_api_id = t.api_id
GROUP BY
    m.season,
    t.id
HAVING
    SUM(m.home_team_goals) < 10
ORDER BY
    t.long_name,
    m.season
```

**Эталонное решение**

```sql
SELECT
    t.long_name,
    m.season,
    SUM(m.home_team_goals) total_goals
FROM
    sql.teams t
    JOIN sql.matches m ON t.api_id = m.home_team_api_id
GROUP BY
    t.id,
    m.season
HAVING
    SUM(m.home_team_goals) < 10
ORDER BY
    1,
    2
```

Сообщение ментору от 10.09.2023

@Андрей_Кукунов_ментор 

Здравствуйте!

**Вопрос по заданию 6.5**

И одновременно, в продолжение дискуссии о  целесообразности намеренных (по Вашему мнению) неточностей (а я бы сказал ненамеренных ошибок) в условиях заданий. 

Изучая этот канал, обратил внимание, что студенты неоднократно обращались за помощью по данному заданию. Выполнив его, понял возможную причину возникающих затруднений.

Вероятнее всего, они обусловлены именно проигнорированными Вами семантическими ошибками в формулировке условия обсуждаемой нами ранее очень похожей задачи, приведенной в качестве примера в учебном материале юнита 4. Очевидно, что по замыслу авторов, осмыслив её, студенты смогут без проблем выполнить задание 6.5.

Если сравнить две формулировки и код решений, то становиться очевидно, что озвученные мною в предыдущем треде ошибки приводят к путанице и непониманию учебного материала.

**Условие задачи из юнита 4**

> Поставим задачу — вывести таблицу с **суммарным количеством забитых голов** в матчах по командам и сезонам для команд, в которых суммарное количество голов в матчах сезона больше 100.


> Альтернативная формулировка1: Поставим задачу: вывести таблицу с суммарным количеством голов за все матчи сезона, проведенные с участием команды, для команд, у которых это значение больше 100.

> Альтернативная формулировка2: Поставим задачу: вывести таблицу со сведенями о сезонах, командах и суммарном количестве голов за сезон в матчах с участием команды, для команд, у которых сумма забитых и пропущенных голов за сезон превышает 100.  

> Альтернативная формулировка3: Поставим задачу: вывести таблицу со сведениями о сезонах, командах и суммарном количестве голов, для команд, у которых сумма голов за все матчи сезона, проведенные с их участием, превышает 100.  


**Условие задачи 6.5** (фрагмент, вызывающий вопросы)

> Напишите запрос, который выведет название команды (`long_name`), сезон (`season`) и **суммарное количество забитых голов** в домашних матчах (`total_goals`).

**Внимание вопрос:** Так что же такое все-таки **"суммарное количество забитых голов"** (применительно к результатам одной команды за сезон), которое требуется подсчитать и отобразить в результирующей таблице в обоих заданиях?

При этом невозможно усомниться в словах авторов о том, что *соединение таблиц — одна из самых сложных тем для освоения и эффективного использования языка SQL*, поэтому, убежден, что никакой дополнительной путаницы в заданих быть не должно априори: формулировки должны быть предельно точными и лаконичными, а уж ошибки в них совершенно недопустимы!

Сообщение от 11.09.2023

Андрей, как и Вы, никакого отношения не имею к среде футбольных болельщиков. 
Но проблема отнюдь не в тонкостях футбольной терминологии, а в языковых конструкциях.

**В условии задания по созданию новой таблицы важно четко и однозначно показать, как содержащиеся в ней признаки соотносятся между собой, особенно, соотношение агрегируемых (числовых) признаков с категориальными**

Исходя из формулировки *"суммарное количество забитых голов"* в условии задания 6.5, **невозможно** достоверно определить источники для агрегации создаваемой таблицы. При описании её агрегируемого признака (количества голов) используется причастие в страдательном залоге (*"забитых"*), употребленное без зависимых слов, что исключает возможность однозначно определить, как этот признак таблицы должен соотноситься с ее категориальным признаком - командой, т.е. **неясно какое именно количество голов надо суммировать: либо которое команда забила, либо пропустила, либо и забила, и пропустила**.

А если к этому винегрету добавить совсем неочевидный для новичков механизм соединения, группировки и агрегации таблиц SQL, то у многих наступает полный коллапс.

Но еще больше сбивает с толку схожее по смыслу задание, разобранное в юните 4, которое по идее должно облегчить выполнение задания 6.5, а получается с точностью до наоборот. 

При описании агрегируемого признака создаваемой таблицы в условии задачи юнита 4 используется та же неоднозначная формулировка (*"суммарное количество забитых голов"*), но с совершенно другим значением. 

Если в задании 6.5 под *суммарным количеством забитых голов* подразумевается **сумма голов, которые команда забила**, то в юните 4 - **сумма голов, которые команда забила и пропустила**.

Не говоря уже о том, что в условии указанной задачи юнита 4 помимо описанной семантической ошибки, которая крайне затрудняет осмысление студентом механизма выполнения инструкций SQL, присутствуют еще и лексическая: придаточное определительное предложение (*"в которых суммарное количество голов в матчах сезона больше 100"*) грамматически не согласовано с определяемым словом (*"команд"*), точнее вообще неясно какое слово оно определяет, т.к. *"голы в командах"* очевидно быть не могут - это больше похоже на абсурд.

Формулировки обоих заданий, очевидно, требуют исправления. 
 
Если не согласны, покажите их филологу, и он Вам гораздо лучше объяснит в чем проблема, и что сбивает с толку.

## Вопросы для самопроверки

---

Все примеры взяты из книги [Нолис Ж., Робинсон Э. Data Science для карьериста. СПб.: Питер, 2021](https://www.litres.ru/zhaklin-nolis/data-science-dlya-karerista/)

### Вопрос 1


*Задание от Лудамилы Джанда (Ludamila Janda), Data Scientist в Amplify*

> Объясните разницу между левым (`LEFT`) и внутренним (`INNER`) соединениями.

**Пример ответа**

**Соединения** — это способы объединения данных из двух разных таблиц (левой и правой) в новую.

Принцип работы соединений заключается в объединении строк между двумя таблицами; набор ключевых столбцов используется для поиска данных в двух одинаковых таблицах, которые необходимо соединить.

В случае левого соединения каждая строка из левой таблицы появляется в полученной таблице, но строки из правой появляются, только если значения в их ключевых столбцах отображаются в левой. При внутреннем соединении обе строки из левой и правой таблиц появляются, только если есть совпадающая строка в другой таблице.

На практике вы можете воспринимать левое соединение как добавление данных из правой таблицы к левой, если она существует (например, с использованием правой таблицы в качестве подстановочной). Внутреннее соединение больше похоже на поиск всех общих данных и создание новой таблицы только из пар.

Обратите внимание, что в нашем варианте ответа мы не упоминали какие‐либо сложности, связанные с появлением повторяющихся строк в данных. Возможно, о них стоит упомянуть, потому что повторяющиеся строки могут повлиять на результат, но с большей долей вероятности эта информация может увести вас от мысли, которую вы пытаетесь донести до интервьюера.


> *Лудамила Джанда любит задавать этот вопрос при наборе кандидатов на роли джуниоров, потому что это не вопрос с подвохом, а минимально необходимые знания. Она считает, что можно многое узнать по тому, как кандидат решает дать ответ. Правильно ответить можно по‐разному: процитировать учебник, используя сложные выражения, или объяснить всё простым языком, не вдаваясь в крайности.*

### Вопрос 2


> Перед вами школьная таблица **TABLE_A** с оценками от 0 до 100, выставленными ученикам нескольких классов.
>
>![](data/dst3-u2-md3_6_1.png)
>
>Как бы вы рассчитали максимальную оценку в каждом классе?

**Пример ответа**

Предполагая, что у нас есть результат предыдущего запроса, сохранённый в **TABLE_B**, мы можем использовать его в этом решении:

```sql
SELECT 
    a.CLASS, 
    a.GRADE, 
    a.STUDENT 
FROM 
    TABLE_A a 
    INNER JOIN TABLE_B b ON a.CLASS = b.CLASS 
    AND a.GRADE = b.GRADE
```

Этот запрос выбирает всех учащихся и их оценки из исходной таблицы **TABLE_A**, где указаны классы и оценки, которые отображаются в таблице максимальных значений **TABLE_B**. Внутреннее соединение работает как фильтр, сохраняя только максимальные комбинации классов/оценок, потому что только в этом случае оценка появляется в таблице **TABLE_B**.

В качестве альтернативы можно было бы использовать подзапрос, чтобы сделать то же самое, не вызывая таблицу **TABLE_B**:

```sql
SELECT 
    a.CLASS, 
    a.GRADE, 
    a.STUDENT 
FROM 
    TABLE_A a 
    INNER JOIN (
        SELECT 
            CLASS, 
            MAX(GRADE) 
        FROM 
            TABLE_A 
        GROUP BY 
            CLASS
    ) b ON a.CLASS = b.CLASS 
    AND a.GRADE = b.GRADE
```

> **Примечание.** Подзапросы мы будем проходить чуть позднее, так что, ознакомившись сейчас, вернитесь к этому варианту, когда изучите синтаксис подзапросов.

Как видите, эту задачу можно решить несколькими способами, но любое решение почти наверняка требует более одного запроса из таблицы **TABLE_A**, поэтому этот вопрос может легко поставить в тупик. На бумаге решение может показаться лёгким, но додуматься до него во время интервью может быть непросто. Даже если вы ответили неправильно, это не значит, что вы провалились.

Решение не даёт каких‐либо специфичных случаев для привязки к максимальному значению. В примере будут возвращены строки для нескольких студентов. Возможно, стоит указать на этот факт интервьюеру, потому что так вы показываете, что обращаете внимание на пограничные случаи.

## Подсчет статистики забитых и пропущенных (мини-проект)

Решается с использованием подзапросов.

```sql
SELECT
    t.api_id,
    t.long_name,
    home_scored.total_home_scored,
    away_scored.total_away_scored,
    home_scored.total_home_scored + away_scored.total_away_scored AS total_goals_scored,
    home_missed.total_home_missed,
    away_missed.total_away_missed,
    home_missed.total_home_missed + away_missed.total_away_missed AS total_goals_missed
FROM
    sql.teams AS t
    JOIN (
        --таблица забитых голов дома
        SELECT
            t.api_id,
            t.long_name,
            SUM(m.home_team_goals) AS total_home_scored
        FROM
            sql.teams AS t
            JOIN sql.matches AS m ON t.api_id = m.home_team_api_id
        GROUP BY
            t.api_id,
            t.long_name
    ) AS home_scored ON t.api_id = home_scored.api_id
    JOIN (
        --таблица забитых голов в гостях
        SELECT
            t.api_id,
            t.long_name,
            SUM(m.away_team_goals) AS total_away_scored
        FROM
            sql.teams AS t
            JOIN sql.matches AS m ON t.api_id = m.away_team_api_id
        GROUP BY
            t.api_id,
            t.long_name
    ) AS away_scored ON t.api_id = away_scored.api_id
    JOIN (
        --таблица пропущенных голов дома
        SELECT
            t.api_id,
            t.long_name,
            SUM(m.away_team_goals) AS total_home_missed
        FROM
            sql.teams AS t
            JOIN sql.matches AS m ON t.api_id = m.home_team_api_id
        GROUP BY
            t.api_id,
            t.long_name
    ) AS home_missed ON t.api_id = home_missed.api_id
    JOIN (
        --таблица пропущенных голов в гостях
        SELECT
            t.api_id,
            t.long_name,
            SUM(m.home_team_goals) AS total_away_missed
        FROM
            sql.teams AS t
            JOIN sql.matches AS m ON t.api_id = m.away_team_api_id
        GROUP BY
            t.api_id,
            t.long_name
    ) AS away_missed ON t.api_id = away_missed.api_id
GROUP BY
    t.api_id,
    t.long_name,
    home_scored.total_home_scored,
    away_scored.total_away_scored,
    home_missed.total_home_missed,
    away_missed.total_away_missed
ORDER BY
    total_goals_scored DESC
```